<a href="https://colab.research.google.com/github/Shubhk0/personalpyprojects/blob/main/Aish_AI_assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



> # First, let's install the required libraries. You can run this cell to install them:
# !pip install google-generativeai SpeechRecognition gtts ipywidgets



In [ ]:
!pip install google-generativeai SpeechRecognition gtts ipywidgets webrtcvad pvporcupine vosk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.7 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=1b5ccada31c6a9f7c09a2ffe7c4360c932aa08b9f48d49b41508fac1404c3d2f
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt


In [ ]:
!pip install vosk --upgrade

In [ ]:
!sudo apt-get install portaudio19-dev python-all-dev python3-all-dev
!pip install PyAudio==0.2.12 --force-reinstall
!pip install gradio


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
portaudio19-dev is already the newest version (19.6.0-1.1).
python-all-dev is already the newest version (2.7.18-3).
python3-all-dev is already the newest version (3.10.6-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Using cached PyAudio-0.2.12-cp310-cp310-linux_x86_64.whl
  Attempting uninstall: PyAudio
    Found existing installation: PyAudio 0.2.12
    Uninstalling PyAudio-0.2.12:
      Successfully uninstalled PyAudio-0.2.12


In [ ]:
from IPython.display import display, Javascript, Audio, HTML
import speech_recognition as sr
from gtts import gTTS
from google.generativeai import GenerativeModel, configure
import os
import threading
import time
import io
import ipywidgets as widgets
import webrtcvad  # For VAD
from vosk import Model, KaldiRecognizer  # For wake word
import urllib.request
import zipfile

# Configure Gemini API
configure(api_key=os.environ.get("GEMINI_API_KEY"))

# Initialize Gemini model
model = GenerativeModel('gemini-pro')

# Initialize speech recognizer
recognizer = sr.Recognizer()

# Initialize VAD
vad = webrtcvad.Vad(3)  # Aggressiveness level (0-3)


# Function to download Vosk model if not found
def download_vosk_model(model_path):
    if not os.path.exists(model_path):
        os.makedirs(model_path, exist_ok=True)
        model_url = "https://alphacephei.com/vosk/models/vosk-model-small-en-us-0.15.zip"
        model_zip_path = os.path.join(model_path, "vosk-model-small-en-us-0.15.zip")
        print("Downloading Vosk model...")
        urllib.request.urlretrieve(model_url, model_zip_path)
        print("Extracting Vosk model...")
        with zipfile.ZipFile(model_zip_path, 'r') as zip_ref:
            zip_ref.extractall(model_path)
        print("Vosk model downloaded and extracted.")

# Download and initialize Vosk model for wake word detection
model_path = "vosk-model"  # Path to store the model
download_vosk_model(model_path)
# Update the model path to point to the extracted model directory
vosk_model_path = os.path.join(model_path, "vosk-model-small-en-us-0.15")
# Initialize the Vosk model with the corrected path
vosk_model = Model(vosk_model_path)
recognizer_vosk = KaldiRecognizer(vosk_model, 16000)  # Assuming 16kHz sample rate



# Define assistant's personality
personality = (
    "You are a friendly and knowledgeable AI assistant named Mitra. "
    "You speak Hindi fluently but can understand and respond in English too. "
    "You have a warm personality and occasionally use Indian cultural references. "
    "You're helpful but also encourage users to think for themselves. "
    "Keep your responses concise and suitable for voice conversation."
)

# Conversation history
conversation_history = []

def generate_response(user_input):
    prompt = f"{personality}\n\nConversation history:\n"
    for entry in conversation_history[-5:]:  # Use recent history for context
        prompt += f"{entry['role']}: {entry['content']}\n"
    prompt += f"Human: {user_input}\nMitra:"

    response = model.generate_content(prompt)
    return response.text

def speak(text):
    tts = gTTS(text=text, lang='hi')
    # Save to in-memory buffer
    fp = io.BytesIO()
    tts.write_to_fp(fp)
    fp.seek(0)
    return fp

def record_audio():
    display(Javascript("""
        const recorder = new MediaRecorder(stream);
        recorder.ondataavailable = (event) => {
            google.colab.kernel.invokeFunction('notebook.record_audio_callback', [event.data], {});
        };
        recorder.start();
        // Removed setTimeout for continuous listening
    """))

def record_audio_callback(audio_data):
    audio_data = io.BytesIO(audio_data)
    audio = sr.AudioData(audio_data.read(), 16000, 2)  # Assuming 16kHz sample rate and 2 channels
    try:
        text = recognizer.recognize_google(audio, language="hi-IN")
        update_conversation("You: " + text)  # Update conversation display

        conversation_history.append({"role": "Human", "content": text})

        response = generate_response(text)
        update_conversation("Mitra: " + response)  # Update conversation display

        conversation_history.append({"role": "Mitra", "content": response})

        audio_output = speak(response)
        display(Audio(audio_output, autoplay=True))
    except sr.UnknownValueError:
        # update_conversation("Mitra: Sorry, I didn't catch that. Could you please repeat?")  # Update conversation display
        pass
    except sr.RequestError:
        update_conversation("Mitra: Sorry, there was an error with the speech recognition service.")  # Update conversation display

# UI elements
start_button = widgets.Button(description="Start Conversation")
output_area = widgets.Output()

def on_button_clicked(b):
    with output_area:
        display(HTML("<h3>Conversation History:</h3>"))
        record_audio()

start_button.on_click(on_button_clicked)

def update_conversation(text):
    with output_area:
        display(HTML("<p>" + text + "</p>"))

# Request microphone access
display(Javascript("""
    navigator.mediaDevices.getUserMedia({ audio: true })
    .then(function(stream) {
        window.stream = stream;
    })
    .catch(function(err) {
        console.error('Error accessing microphone:', err);
    });
"""))

# Display UI elements
display(HTML("<h1>Jupyter Voice Assistant</h1>"))
display(start_button, output_area)

# Continuous listening loop (runs in a separate thread)
# Continuous listening loop (runs in a separate thread)
def continuous_listening():
    # Initialize PyAudio
    p = pyaudio.PyAudio()

    # Open audio stream
    stream = p.open(format=pyaudio.paInt16,
                    channels=1,
                    rate=16000,
                    input=True,
                    frames_per_buffer=8000)

    while True:
        # Read audio data from the microphone
        audio_data = stream.read(8000)

        # Check for wake word using Vosk
        if recognizer_vosk.AcceptWaveform(audio_data):
            result = recognizer_vosk.Result()
            if "computer" in result:  # Replace "computer" with your wake word
                # Wake word detected, start recording
                record_audio()
                break  # Exit the loop after recording

    # Stop and close the audio stream
    stream.stop_stream()
    stream.close()
    p.terminate()


# Start continuous listening in a separate thread
listening_thread = threading.Thread(target=continuous_listening)
listening_thread.start()

<IPython.core.display.Javascript object>

Button(description='Start Conversation', style=ButtonStyle())

Output()

Exception in thread Thread-23 (continuous_listening):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-26-ae86fa931d9f>", line 147, in continuous_listening
NameError: name 'pyaudio' is not defined. Did you mean: 'Audio'?
